## Sudoku solver

Here I try to make a program that solves sudoku.<br>
It takes as input a sudoku grid, entered row-by-row, with 0s for spaces (program stops if user types 'stop')<br>
It returns the number of solutions given that grid, and one possible solution<br>
<br>
<b>example</b><br>
&emsp;<i>input</i><br>
&emsp;row 1: 030010060<br>
&emsp;row 2: 750030048<br>
&emsp;row 3: 006984300<br>
&emsp;row 4: 003000800<br>
&emsp;row 5: 912000674<br>
&emsp;row 6: 004000500<br>
&emsp;row 7: 001675200<br>
&emsp;row 8: 680090005<br>
&emsp;row 9: 090040030<br>
<br>
&emsp;<i>output</i><br>
&emsp;Number of solutions: 5<br>
&emsp;Here's one solution: <br>
&emsp;[4, 3, 8, 5, 1, 7, 9, 6, 2]<br>
&emsp;[7, 5, 9, 2, 3, 6, 1, 4, 8]<br>
&emsp;[1, 2, 6, 9, 8, 4, 3, 5, 7]<br>
&emsp;[5, 7, 3, 4, 6, 1, 8, 2, 9]<br>
&emsp;[9, 1, 2, 8, 5, 3, 6, 7, 4]<br>
&emsp;[8, 6, 4, 7, 2, 9, 5, 1, 3]<br>
&emsp;[3, 4, 1, 6, 7, 5, 2, 8, 9]<br>
&emsp;[6, 8, 7, 3, 9, 2, 4, 1, 5]<br>
&emsp;[2, 9, 5, 1, 4, 8, 7, 3, 6]<br><br>In future work, I would like to implement this program with a GUI<br>


In [5]:
import copy

# the main solver function. it takes a board as input and returns an array of possible solutions given that board
def solve(board):
    
    # base case: if board is already filled, return array contains it iff it is solved
    if filled(board): 
        if ok(board): return [board]
        return []
    
    # initialise return array
    possibles = []
    
    # try some human methods to speed things up
    human(board)
    
    # find the first empty slot in the map
    row = 0
    while (board[row].count(0) == 0): row+=1
    col = board[row].index(0)
    
    # try each possible number, and return array of solve() wherever there is a solution
    possibles = []
    
    for num in range(1,10):
        if (canput(board,col,row,num)): 
            temp = copy.deepcopy(board)
            temp[row][col] = num
            human(temp)
            possibles += solve(temp)
                    
    return possibles

# check if board is filled
def filled(board): return sum([row.count(0) for row in board]) == 0
    
# check if filled board is error-free
def ok(board):
    for i in range(9):
        if (sorted(board[i]) != list(range(1,10))): return False
        if (sorted(board[:][i]) != list(range(1,10))): return False
        box = [board[(i//3)*3][(i%3)*3 + j] for j in range(3)] + [board[(i//3)*3+1][(i%3)*3 + j] for j in range(3)] + [board[(i//3)*3+2][(i%3)*3 + j] for j in range(3)]
        if (sorted(box) != list(range(1,10))): return False
    return True
            
# human methods
def human(board):
    canhuman = 1
    while canhuman:
        canhuman = 0
        for i in range(9):
            # solve a row with 8 numbers filled
            if (board[i].count(0) == 1): 
                board[i][board[i].index(0)] = [j for j in range(1,10) if j not in board[i]][0]
                canhuman = 1
            
            # solve a column with 8 numbers filled
            if (board[:][i].count(0) == 1): 
                board[board[:][i].index(0)][i] = [j for j in range(1,10) if j not in board[:][i]][0]
                canhuman = 1
            
            # solve a box with 8 numbers filled
            box = [board[(i//3)*3][(i%3)*3 + j] for j in range(3)] + [board[(i//3)*3+1][(i%3)*3 + j] for j in range(3)] + [board[(i//3)*3+2][(i%3)*3 + j] for j in range(3)]
            if (box.count(0) == 1): 
                board[(i//3)*3+box.index(0)//3][(i%3)*3+box.index(0)%3] = [j for j in range(1,10) if j not in box][0]
                canhuman = 1

# check if, for a given board, at a row and column, a number can be put
def canput(board,col,row,num):
    # check for clash with previous numbers
    if not safe(board,col,row,num): return False
    
    # check for solvability after the number is put
    if cansolve(board,col,row,num): return True
    return False

# check if, for a given board, at a row and column, inserting a number causes a clash anywhere
def safe(board,col,row,num):
    # scan every other member of that row
    for r in range(9):
        if ((board[r][col] == num) and (r!=row)): return False
    
    # scan every other member of that column
    for c in range(9):
        if ((board[row][c] == num) and (c!=col)): return False
    
    # scan every other member of that box
    for b in range(9):
        rtar = (row//3)*3 + (b//3)
        ctar = (col//3)*3 + (b%3)
        if ((board[rtar][ctar]==num) and ((row,col)!=(rtar,ctar))): return False
        
    return True
    
# check if, for a given board, at a row and column, inserting a number results in a solvable map
def cansolve(board,col,row,num):
    # generate the board in question
    temp = copy.deepcopy(board)
    temp[row][col] = num
    
    # try some human methods
    human(temp)
    
    # base case: if filled, just say whether its solved
    if filled(temp): return ok(temp)
    
    # find the next empty slot in the map
    row = 0
    while (temp[row].count(0) == 0): row+=1
    col = temp[row].index(0)
    
    # check if some number works
    for i in range(1,10):
        if canput(temp,col,row,i): return True
    return False

# ask for input
board = []
for i in range(9):
    # validation
    valid = 0
    
    while not valid:
        valid = 1
        temp = input('row {}: '.format(i+1))
        if (temp == 'stop'): print('ok'); valid = 2; temp = '111111111'
        if (len(temp) != 9): print('check input length'); valid = 0; temp = '1'
        if not temp.isnumeric(): print('only enter numbers'); valid = 0
    if (valid == 2): board = ''; break
    ans = [int(char) for char in temp]
    board.append(ans)

# # for testing
# board = [
#     [0,3,0,0,1,0,0,6,0],
#     [7,5,0,0,3,0,0,4,8],
#     [0,0,6,9,8,4,3,0,0],
#     [0,0,3,0,0,0,8,0,0],
#     [9,1,2,0,0,0,6,7,4],
#     [0,0,4,0,0,0,5,0,0],
#     [0,0,1,6,7,5,2,0,0],
#     [6,8,0,0,9,0,0,0,5],
#     [0,9,0,0,4,0,0,3,0]
# ]

if board:
    # echo input
    print('\ninput')
    for row in board: print(row)

    # call solve function
    arr = solve(board)

    # if no solution, say so
    if (len(arr) == 0): print('\nNo solution')
    else:
        print('\nNumber of solutions: {}'.format(len(arr)))
        print('Here\'s one solution: ')
        for row in arr[0]: print(row)



input
[0, 3, 0, 0, 1, 0, 0, 6, 0]
[7, 5, 0, 0, 3, 0, 0, 4, 8]
[0, 0, 6, 9, 8, 4, 3, 0, 0]
[0, 0, 3, 0, 0, 0, 8, 0, 0]
[9, 1, 2, 0, 0, 0, 6, 7, 4]
[0, 0, 4, 0, 0, 0, 5, 0, 0]
[0, 0, 1, 6, 7, 5, 2, 0, 0]
[6, 8, 0, 0, 9, 0, 0, 0, 5]
[0, 9, 0, 0, 4, 0, 0, 3, 0]

Number of solutions: 5
Here's one solution: 
[4, 3, 8, 5, 1, 7, 9, 6, 2]
[7, 5, 9, 2, 3, 6, 1, 4, 8]
[1, 2, 6, 9, 8, 4, 3, 5, 7]
[5, 7, 3, 4, 6, 1, 8, 2, 9]
[9, 1, 2, 8, 5, 3, 6, 7, 4]
[8, 6, 4, 7, 2, 9, 5, 1, 3]
[3, 4, 1, 6, 7, 5, 2, 8, 9]
[6, 8, 7, 3, 9, 2, 4, 1, 5]
[2, 9, 5, 1, 4, 8, 7, 3, 6]
